# Visualization benchmark data

In [ ]:
# Imports
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Requesting data
from parsing import parse_results_file

cases = parse_results_file("../result.jsonl")
print(cases)

## Latency

### Bar plot with error bars

In [ ]:
# Extract values
x = [case.nr_resources for case in cases]
get_latest = [case.duration_get_latest for case in cases]
direct = [case.duration_direct for case in cases]

mean_get_latest = np.mean(get_latest, axis=1)
std_get_latest = np.std(get_latest, axis=1)
mean_direct = np.mean(direct, axis = 1)
std_direct = np.std(direct, axis = 1)

In [ ]:
# Create a bar plot with error bars to compare average durations

width = 0.35  # width of the bars
x_pos = np.arange(len(x))  # bar positions

fig, ax = plt.subplots(figsize=(10, 6))

# Plot bars for get_latest and direct durations
bars1 = ax.bar(
    x_pos - width / 2,
    mean_get_latest,
    width,
    yerr=std_get_latest,
    capsize=5,
    label="get_latest",
    color="blue",
    alpha=0.7,
)
bars2 = ax.bar(
    x_pos + width / 2,
    mean_direct,
    width,
    yerr=std_direct,
    capsize=5,
    label="direct",
    color="green",
    alpha=0.7,
)

# Labeling
ax.set_xlabel("Number of Resources")
ax.set_ylabel("Duration (µs)")
ax.set_xticks(x_pos)
ax.set_xticklabels(x)
ax.legend()
ax.grid(True, axis="y", linestyle="--", alpha=0.6)

plt.tight_layout()
plt.show()